In [1]:
import numpy as np
import random
import csv

In [6]:
def create_board():
    return np.zeros((6, 7), dtype=int)

def is_valid_move(board, col):
    return board[0][col] == 0

def get_next_open_row(board, col):
    for row in range(5, -1, -1):
        if board[row][col] == 0:
            return row

def drop_piece(board, row, col, piece):
    board[row][col] = piece

def game_over(board):
    return check_winner(board) is not None or np.all(board != 0)

def check_winner(board):
    # Check horizontal locations
    for row in range(6):
        for col in range(4):  # Only start checks up to column 4
            if board[row, col] != 0 and all(board[row, col + i] == board[row, col] for i in range(4)):
                return board[row, col]

    # Check vertical locations
    for row in range(3):  # Only start checks up to row 3
        for col in range(7):
            if board[row, col] != 0 and all(board[row + i, col] == board[row, col] for i in range(4)):
                return board[row, col]

    # Check positively sloped diagonals
    for row in range(3):
        for col in range(4):
            if board[row, col] != 0 and all(board[row + i, col + i] == board[row, col] for i in range(4)):
                return board[row, col]

    # Check negatively sloped diagonals
    for row in range(3):
        for col in range(4):
            if board[row + 3 - col, col] != 0 and all(board[row + i, col + i] == board[row, col] for i in range(4)):
                return board[row, col]

    return None

def evaluate_board(board):
    winner = check_winner(board)
    if winner == 1:
        return 1  # Player 1 wins
    elif winner == -1:
        return -1  # Player -1 wins
    else:
        return 0  # No winner or draw



In [7]:
def minimax(board, depth, maximizing_player):
    if depth == 0 or game_over(board):
        return evaluate_board(board)

    valid_moves = [c for c in range(7) if is_valid_move(board, c)]
    if maximizing_player:
        value = -float('inf')
        for col in valid_moves:
            row = get_next_open_row(board, col)
            temp_board = board.copy()
            drop_piece(temp_board, row, col, 1)
            value = max(value, minimax(temp_board, depth - 1, False))
        return value
    else:
        value = float('inf')
        for col in valid_moves:
            row = get_next_open_row(board, col)
            temp_board = board.copy()
            drop_piece(temp_board, row, col, -1)
            value = min(value, minimax(temp_board, depth - 1, True))
        return value

def get_best_move(board, depth=4):
    valid_moves = [c for c in range(7) if is_valid_move(board, c)]
    best_move = None
    best_value = -float('inf')
    for col in valid_moves:
        row = get_next_open_row(board, col)
        temp_board = board.copy()
        drop_piece(temp_board, row, col, 1)
        move_value = minimax(temp_board, depth - 1, False)
        if move_value > best_value:
            best_value = move_value
            best_move = col
    return best_move


In [8]:
def generate_dataset(num_samples):
    dataset = []
    for _ in range(num_samples):
        board = create_board()
        moves = random.randint(5, 20)  # Random number of moves
        for move in range(moves):
            valid_moves = [c for c in range(7) if is_valid_move(board, c)]
            if not valid_moves:
                break
            col = random.choice(valid_moves)
            row = get_next_open_row(board, col)
            piece = 1 if move % 2 == 0 else -1
            drop_piece(board, row, col, piece)

        best_move = get_best_move(board)
        if best_move is not None:
            flattened_board = board.flatten().tolist()
            dataset.append(flattened_board + [best_move])

    # Save to CSV
    with open('connect4_dataset.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        for row in dataset:
            writer.writerow(row)


In [9]:
generate_dataset(10000)